In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os, shutil
from PIL import Image 
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [3]:
idg = ImageDataGenerator(validation_split=0.1, horizontal_flip=True,
                         brightness_range=[0.2, 0.4],
                        preprocessing_function=preprocess_input)

In [4]:
labels = pd.read_csv('../input/dog-breed-identification/labels.csv')
labels.head()

In [5]:
labels['id'] = labels['id'] + 'jpg'
labels.head()

In [9]:
train_generator = idg.flow_from_dataframe(labels, 
                                         '../input/dog-breed-identification/train/',
                                         x_col = 'id', 
                                         y_col = 'breed',
                                         target_size=(224, 224),
                                         batch_size = 32, 
                                         subset='training', 
                                         validate_filenames=True)

val_generator = idg.flow_from_dataframe(labels, 
                                       '../input/dog-breed-identification/train/',
                                        x_col='id', 
                                        y_col='breed', 
                                        target_size=(224, 224),
                                        batch_size = 32, 
                                        subset='validation',
                                        validate_filenames=True)


In [14]:
vgg_model = VGG16(include_top=False, input_shape=(224,224,3))
vgg_model.summary()

In [15]:
for layer in vgg_model.layers:
    layer.trainable=False

In [17]:
vgg_model.summary()

In [19]:
for layer in vgg_model.layers:
    print(f'layer={layer.name} is_trainable: {layer.trainable}')

In [20]:
vgg_output = vgg_model.output

In [21]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [27]:
flatten_output = Flatten()(vgg_output)

dense_output = Dense(512, activation=relu)(flatten_output)

dense_output2 = Dense(120, activation=softmax)(dense_output)


In [28]:
final_model = Model(inputs=vgg_model.input, outputs=dense_output2)

In [29]:
final_model.summary()

In [30]:
for layer in final_model.layers:
    print(f'layer={layer.name} is_trainable: {layer.trainable}')

In [31]:
final_model.compile(optimizer=SGD(), 
                    loss=categorical_crossentropy,
                   metrics=['acc'])

In [ ]:
# ! ps -aux | grep tensorboard
# ! kill -9 2925

In [35]:
from tensorflow.keras.callbacks import TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs

# %reload_ext tensorboard


In [36]:
tfb=TensorBoard("logs")

In [37]:
final_model_history = final_model.fit_generator(train_generator,
                         epochs=30, validation_data=val_generator, callbacks=[tfb])